In [ ]:
#pip install cvrplib

In [4]:
from random import randrange, random, seed
from math import exp
import glob, os
import cvrplib
import sys
from datetime import datetime

MAX_COUNT = 10000000

# Считаем суммарную Capacity для маршрута
def calc_cap(route, cd):
    sum = 0
    for i in range(1,len(route)):
        sum = sum + cd[route[i]]
    return sum

# Считаем суммарную длину маршрута
def total_length(cr,var):
    sum = 0
    for r in range(len(cr)): # r - маршрут;
        for i in range(len(cr[r])): # i - клиенты;
            if cr[r][i+1] == 0:
                break
            # Сумма евклидовых расстояний всех маршрутов
            sum = sum + var.distances[cr[r][i]][cr[r][i+1]]
    return sum

# Смотрим индекс последнего клиента на маршруте
def last_client(route):
    i = 1
    while route[i] != 0:
        i += 1
    return i

def show_routes(cr,goal,result):
    for r in range(len(cr)): # Вывод результата
        print ("Route #",r+1, sep='', end=': ') 
        for i in range(1,len(cr[r])):
            if cr[r][i] != 0:
                print(cr[r][i], end=' ')
        print()
    if len2 > goal:
        print( "FAILED: ", )
    print( "Cost", len2 )

# Меняем два элемента (клиентов) местами
def cswap(cr,var,T,change):
    # Текущая длина маршрутов
    len1 = total_length( cr, var )
    # Создаём новый массив
    cr2 = [0] * len(cr)
    for i in range(len(cr)):
        cr2[i] = [0] * var.dimension
    # Поэлементно копируем весь маршрут на случай необходимости отката
    for i in range(len(cr)):
        for j in range(var.dimension):
            cr2[i][j] = cr[i][j]
    for k in range(change):
        r1 = randrange(len(cr))
        r2 = randrange(len(cr))
        # Последние колонки в маршрутах
        lc1 = last_client(cr[r1])+1
        lc2 = last_client(cr[r2])+1
        # Выбираем рандомных клиентов из маршрутов (может быть и в рамках одного маршрута)
        i1 = randrange(1,lc1)
        i2 = randrange(1,lc2)

        if( (cr[r1][i1] != 0) & (cr[r2][i2] != 0) ): # Меняем клиента на клиента, если в маршруте есть хотя бы по 1 клиенту
            if cr[r1][i1] != cr[r2][i2]:
                swap = cr[r1][i1]
                cr[r1][i1] = cr[r2][i2]
                cr[r2][i2] = swap
                #len2 = total_length( cr, var )
        else: # Перебрасываем клиента с маршрута на маршрут
            if (cr[r1][i1] + cr[r2][i2]) != 0:
                if( r1 != r2 ): # Действие имеет смысл только между разными маршрутами
                    # Ищем пустой маршрут среди двух
                    if cr[r1][i1] != 0:
                        # Если первый элемент не пустой, меняем ячейки местами
                        k = r1
                        r1 = r2
                        r2 = k
                        k = i1
                        i1 = i2
                        i2 = k
                        k = lc1
                        lc1 = lc2
                        lc2 = k
                    k = cr[r1][i1]
                    cr[r1][i1] = cr[r2][i2]
                    i1 = randrange(1,lc1) # Выбираем позицию в маршруте для нового клиента
                    while k != 0: # Вставляем клиента в маршрут и сдвигаем существующих клиентов вправо
                        i1 += 1
                        l = cr[r1][i1]
                        cr[r1][i1] = k
                        k = l
                    i = i2 + 1
                    while cr[r2][i] != 0: # Перемещаем клиентов влево
                        cr[r2][i-1] = cr[r2][i]
                        cr[r2][i] = 0
                        i += 1
    ret = 1 # Изначально считаем, что перестановка удалась
    for r in range(len(cr)): 
        if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, не превысили ли мы вместимость 
            ret = 0
            break
    if ret == 1: 
        len2 = total_length(cr,var)
        if (len2 > len1): # Если длина стала больше, по экспоненте определяем вероятность перестановки
            p = exp((len1-len2)/T)
            q = random()
            if p <= q:
                ret = 0 # Отмена перестановки
    if ret == 0:
        for i in range(len(cr)):
            for j in range(var.dimension):
                cr[i][j] = cr2[i][j]
    return ret # 0 - неудачный обмен, 1 - удачный обмен

entries = glob.glob('datafiles\\*.vrp')
for file in entries:
    var = cvrplib.read(file)
    trucks = int(var.name[var.name.find('-k')+2:])
    fr = open(file)
    skip = fr.readline()    # NAME
    val = fr.readline()     # COMMENT
    data = val.split(" ")
    goal = int(data[len(data)-1][:-2])
    fr.close()

    seed(0)
    cr = [0] * trucks       # Массив с маршрутами
    ratio = float(sum(var.demands)/(trucks*var.capacity))
    print()
    print("Name:", var.name, " Trucks:", trucks, " Cap:", var.capacity, " var.dimension:",
          var.dimension, " Goal:", goal, " Sum:", sum(var.demands),
          " Ratio:", ratio, sep='')
    start_time = datetime.now()
    for i in range(trucks):
        cr[i] = [0] * var.dimension

    # В случае неуспеха повторяем попытку с другим начальным распределением
    distr = False
    while distr == False:
        for r in range(trucks): # Обнуляем массив
            for i in range(var.dimension):
                cr[r][i] = 0
        # Случайное распределение клиентов по маршрутам с учетом вместимости
        for i in range(1,var.dimension):
            ok = False # i-тый клиент получил свой маршртут
            r = -1 # Признак того, что маршрут ещё не был присвоен
            '''
            Возможна ситуация, когда очередного клиента невозможно поместить в очередной маршрут.
            В этом случае нужно начать распределение заново.
            '''
            while ok == False:
                if r == -1:
                    r = randrange(trucks)
                    fail = 0
                else:
                    r += 1
                    r = r % trucks
                j = last_client(cr[r])
                cr[r][j] = i
                if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, помещается ли клиент в маршрут
                    cr[r][j] = 0
                    fail += 1
                    if fail > trucks:
                        i = 999
                        break
                else:
                    ok = True
        if i != 999:
            distr = True

    T = total_length( cr, var ) # Начальная "температура"
    print("T =",T)
    for count in range(1,MAX_COUNT):
        # Меняем 1 или 2 маршрута, определяется рандомно
        cswap(cr,var,0.002*(MAX_COUNT-count)/MAX_COUNT*goal,randrange(1,3))
        len2 = total_length( cr, var )
        if len2 <= goal:
            break;
    end_time = datetime.now()
    show_routes(cr,goal,len2)
    print( "Calculation time:", end_time - start_time )


Name:A-n32-k5 Trucks:5 Cap:100 var.dimension:32 Goal:784 Sum:410 Ratio:0.82
T = 1863
Route #1: 7 13 2 3 23 26 26 26 26 26 26 16 
Route #2: 18 8 28 4 11 
Route #3: 9 22 15 5 27 24 14 
Route #4: 30 12 1 21 31 19 
Route #5: 20 25 10 29 6 17 19 
Cost 784
Calculation time: 0:00:02.775577

Name:A-n33-k5 Trucks:5 Cap:100 var.dimension:33 Goal:661 Sum:446 Ratio:0.892
T = 1828
Route #1: 24 6 1 31 11 19 14 21 
Route #2: 23 4 12 10 17 9 
Route #3: 2 32 32 22 18 28 3 
Route #4: 20 13 8 5 26 7 
Route #5: 15 30 25 27 16 29 
Cost 660
Calculation time: 0:00:00.123673

Name:A-n33-k6 Trucks:6 Cap:100 var.dimension:33 Goal:742 Sum:541 Ratio:0.9016666666666666
T = 1334
Route #1: 15 20 17 25 30 27 16 
Route #2: 32 10 12 
Route #3: 1 18 7 6 19 
Route #4: 14 4 26 2 9 
Route #5: 22 13 5 3 8 24 23 31 
Route #6: 28 21 11 29 
Cost 728
Calculation time: 0:00:00.057881

Name:A-n34-k5 Trucks:5 Cap:100 var.dimension:34 Goal:778 Sum:460 Ratio:0.92
T = 1736
Route #1: 10 13 21 32 12 9 16 2 
Route #2: 5 30 24 29 14 4 


Route #1: 5 45 2 42 32 53 1 12 13 19 
Route #2: 51 43 3 14 22 34 46 39 
Route #3: 44 56 40 59 37 4 
Route #4: 62 28 23 7 54 
Route #5: 64 26 9 15 27 49 18 33 
Route #6: 17 47 31 6 11 48 57 
Route #7: 29 55 30 35 36 
Route #8: 41 50 16 38 61 58 20 
Route #9: 21 25 24 52 8 10 
Cost 1172
Calculation time: 0:00:02.079446

Name:A-n69-k9 Trucks:9 Cap:100 var.dimension:69 Goal:1159 Sum:845 Ratio:0.9388888888888889
T = 3546
Route #1: 40 39 56 8 20 35 45 
Route #2: 16 48 62 22 31 18 2 
Route #3: 23 51 30 53 33 49 
Route #4: 12 29 52 14 47 4 60 
Route #5: 38 68 13 44 15 3 6 9 
Route #6: 27 65 55 25 37 32 10 
Route #7: 19 24 43 50 17 1 36 
Route #8: 64 61 21 7 11 63 46 5 
Route #9: 28 34 42 26 54 57 58 66 41 59 
Cost 1159
Calculation time: 0:00:02.760578

Name:A-n80-k10 Trucks:10 Cap:100 var.dimension:80 Goal:1763 Sum:942 Ratio:0.942
T = 4602
Route #1: 13 70 76 3 29 12 52 14 48 
Route #2: 74 30 37 8 78 77 51 67 38 58 72 
Route #3: 50 45 4 32 22 54 33 64 
Route #4: 10 28 79 59 53 66 
Route #5: 63 

Route #1: 9 61 62 57 3 30 
Route #2: 16 38 50 46 34 6 
Route #3: 25 21 10 65 32 37 23 
Route #4: 48 63 56 15 58 5 4 14 
Route #5: 49 1 35 19 40 64 60 
Route #6: 44 42 36 53 39 
Route #7: 31 33 2 41 29 59 24 
Route #8: 52 45 43 27 51 8 
Route #9: 28 13 7 20 11 36 
Route #10: 17 66 26 18 47 12 54 55 22 
Cost 995
Calculation time: 0:00:02.385585

Name:B-n68-k9 Trucks:9 Cap:100 var.dimension:68 Goal:1272 Sum:837 Ratio:0.93
T = 3470
Route #1: 30 21 20 41 8 33 25 11 11 39 3 
Route #2: 35 47 42 24 18 10 
Route #3: 19 50 38 55 67 36 29 6 
Route #4: 7 61 49 
Route #5: 34 48 26 52 17 16 44 
Route #6: 64 27 66 65 43 13 5 62 
Route #7: 4 1 37 2 57 51 22 28 
Route #8: 56 46 14 54 12 59 9 45 
Route #9: 15 63 40 53 23 31 60 58 32 
Cost 1272
Calculation time: 0:00:01.343407

Name:B-n78-k10 Trucks:10 Cap:100 var.dimension:78 Goal:1221 Sum:937 Ratio:0.937
T = 4009
Route #1: 1 52 21 69 67 6 41 14 
Route #2: 72 43 24 38 4 59 35 
Route #3: 17 58 65 75 57 7 76 32 66 60 
Route #4: 56 71 12 34 45 46 31 73 
Ro